### Basic global settings trials including ncluster, method and data used

In [1]:
import json
import itertools
from functions import *
%set_env GUROBI_HOME=/usr/local/gurobi/linux64

env: GUROBI_HOME=/usr/local/gurobi/linux64


In [2]:
with open('config.json', 'r') as f:
    config_all = json.loads(f.read())

config = config_all['Texas_7_years_0']
inputs = config['inputs']
settings = config['settings']
print(config)
pv_all = np.genfromtxt(
    os.path.join(config['root'], config['pv_path']), delimiter=',', filling_values=0)
wind_all = np.genfromtxt(
    os.path.join(config['root'], config['wind_path']), delimiter=',', filling_values=0)
load_all = np.genfromtxt(
    os.path.join(config['root'], config['load_path']), delimiter=',', filling_values=0)  # Data load

{'root': '/home/jupyter-zyh/Gnhe/data/Texas/', 'save_root': '/home/jupyter-zyh/Gnhe/analysis_profile/global_setting/Integer/Texas/profile-4/', 'pv_path': 'pv_data_Texas.csv', 'wind_path': 'wind_data_Texas.csv', 'load_path': 'power_demand_Texas.csv', 'note_1': 'This property is set to indicate what are above properties for path. No need to change.', 'inputs': {'c_bat': 200000, 'c_bat_power': 70000, 'c_pv': 1000000, 'c_wind': 1500000, 'c_gen_inv': 1000000, 'c_gen_a': 0.1, 'c_gen_b': 30, 'up_time': 6, 'down_time': 6, 'energy_cap': 200, 'power_cap': 200, 'gen_cap': 1, 'mdc': 50, 'effi': 0.9, 'rps': 0.8, 'note_2': 'This property is set to indicate what are above properties for model. Change with model parameters.', 'solver': 1, 'print_log': 0, 'mipgap': 0.001, 'timelimit': 210240, 'note_3': 'This property is set to indicate what are above properties for solver. No need to change.'}, 'settings': {'data': 'Texas', 'profile_id': 4, 'day_num': 1092, 'period': 168, 'normalize': False, 'feature_s

In [3]:
day_num = settings['day_num']
time_set = np.arange(24*day_num)  # Time horizon
if 'profile_id' in settings:
    profile_id = settings['profile_id']
    renewable = [wind_all[time_set, profile_id],pv_all[time_set, profile_id]]
else:
    # Denote that first profile is used and no other profile exists.
    profile_id = -1
    renewable = [wind_all[time_set], pv_all[time_set]]
load = load_all[time_set]*100  # Data extract

In [4]:
for rps in [0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]: 
    print("Rps = " + str(rps))
    inputs['print_log'] = 1
    inputs['rps'] = rps
    config['inputs'] = inputs
    
    # Full time model solve
    start_time = time.time()
    m = time_model_solve(inputs, renewable, load)
    elapsed_time = time.time() - start_time
    results = dict()
    for v in m.component_objects(Var, active=True):
        var_object = getattr(m, str(v))
        results[str(v)] = []
        if var_object.is_indexed() and len(var_object) >= 1:
            for t in range(len(var_object)):
                results[str(v)].append(var_object[t].value)
        elif len(var_object) == 1:
            results[str(v)] = var_object.value

    # Benchmark calculation
    to_add = ['renewable_cap', 'N', 'max_energy', 'max_power', 'total_cost'] if inputs['gen_cap'] else [
    'renewable_cap', 'max_energy', 'max_power', 'total_cost']
    expected = {v: results[v] for v in to_add}
    expected['description'] = 'benchmark'
    expected['elapsed_time'] = elapsed_time
    for i in range(len(renewable)):
        expected['renewable_cap_{}'.format(i)] = results['renewable_cap'][i]
    pd.DataFrame([expected]).to_csv('/home/jupyter-zyh/Gnhe/benchmark/Integer/Texas/profile-{}/expected/expected_{}_rps_{}_1e-3.csv'.format(profile_id,day_num,rps))
    print("Expectation : Done.")
    inputs['print_log'] = 0
    config['inputs'] = inputs
    sim_features_df = sim_features(config,wind_all,pv_all,load_all)
    sim_features_df.to_csv('/home/jupyter-zyh/Gnhe/benchmark/Integer/Texas/profile-{}/features/features_{}_rps_{}_1e-3.csv'.format(profile_id,day_num,rps))
    print("Simulation : Done.")
    df = run_trials(config,wind_all,pv_all,load_all,expected,sim_features_df[['renewable_cap_0','renewable_cap_1', 'N', 'max_energy', 'max_power']])
    df.set_index('description').to_csv(
        config['save_root']+settings['data']+('_Int_' if inputs['gen_cap'] else '_')+('Renewable'+str(profile_id)+'_' if 'profile_id' in settings else '_')+str(settings['day_num'])+'Days_'+str(settings['period'])+'HrsPerPeriod_{}.csv'.format(rps))
print("All Done!")

Rps = 0.5
Academic license - for non-commercial use only - expires 2021-07-30
Using license file /home/jupyter-zyh/gurobi.lic
Read LP format model from file /tmp/tmpx11g468v.pyomo.lp
Reading time = 1.58 seconds
x209675: 471751 rows, 209675 columns, 1351329 nonzeros
Changed value of parameter timelimit to 210240.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter mipgap to 0.001
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 28 physical cores, 56 logical processors, using up to 28 threads
Optimize a model with 471751 rows, 209675 columns and 1351329 nonzeros
Model fingerprint: 0x549df9b5
Variable types: 131050 continuous, 78625 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-07, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+02]
         Consider reformulating model or setting NumericFocus parameter
         to a

     0     0 2.0370e+08    0 31415 2.0508e+08 2.0370e+08  0.67%     - 1912s
     0     0 2.0370e+08    0 31355 2.0508e+08 2.0370e+08  0.67%     - 2220s
H    0     0                    2.041991e+08 2.0370e+08  0.25%     - 2421s
     0     0 2.0370e+08    0 31354 2.0420e+08 2.0370e+08  0.25%     - 2428s
     0     0 2.0370e+08    0 31279 2.0420e+08 2.0370e+08  0.25%     - 2498s
     0     0 2.0370e+08    0 31221 2.0420e+08 2.0370e+08  0.25%     - 2702s
H    0     0                    2.041987e+08 2.0370e+08  0.25%     - 2843s
     0     2 2.0370e+08    0 31173 2.0420e+08 2.0370e+08  0.25%     - 2914s
     1     4 2.0384e+08    1 29233 2.0420e+08 2.0370e+08  0.25%  3014 2954s
     3     8 2.0384e+08    2 29232 2.0420e+08 2.0370e+08  0.25%  1078 2965s
     7    16 2.0384e+08    3 29225 2.0420e+08 2.0370e+08  0.25%   491 2971s
    15    32 2.0384e+08    4 29223 2.0420e+08 2.0370e+08  0.25%   231 2983s
    31    60 2.0384e+08    5 29211 2.0420e+08 2.0370e+08  0.25%   118 3335s
    59    88 2

   64525    4.7970703e+08   0.000000e+00   2.863144e+08    105s
   65485    4.7070754e+08   0.000000e+00   3.409990e+08    110s
   66445    4.6426352e+08   0.000000e+00   4.031275e+08    116s
   67405    4.5863732e+08   0.000000e+00   6.082831e+08    121s
   68205    4.5462855e+08   0.000000e+00   1.734097e+09    125s
   69005    4.5052578e+08   0.000000e+00   6.528389e+08    130s
   69965    4.4583884e+08   0.000000e+00   2.132712e+08    136s
   70765    4.4224335e+08   0.000000e+00   1.137396e+09    141s
   71565    4.3814495e+08   0.000000e+00   8.693499e+08    146s
   72365    4.3426551e+08   0.000000e+00   3.120703e+08    151s
   73165    4.3117956e+08   0.000000e+00   8.126765e+08    156s
   73965    4.2848107e+08   0.000000e+00   3.061302e+08    161s
   74605    4.2632315e+08   0.000000e+00   4.650588e+08    165s
   75405    4.2418560e+08   0.000000e+00   4.899014e+08    171s
   76045    4.2209831e+08   0.000000e+00   2.347546e+08    175s
   76845    4.1975775e+08   0.000000e+00

10th trial:  7.865175485610962
11th trial:  10.289945363998413
12th trial:  4.50634241104126
13th trial:  9.293037176132202
14th trial:  8.137896299362183
15th trial:  4.602616548538208
16th trial:  9.399489879608154
17th trial:  8.537742137908936
18th trial:  23.463212251663208
19th trial:  9.735793113708496
20th trial:  10.311739921569824
21th trial:  16.64069390296936
22th trial:  7.925470590591431
23th trial:  15.064491748809814
24th trial:  14.789530754089355
25th trial:  11.180738687515259
26th trial:  12.642525911331177
27th trial:  6.233417749404907
28th trial:  20.299309015274048
29th trial:  13.915141582489014
30th trial:  12.444435119628906
31th trial:  13.741777896881104
32th trial:  13.310669183731079
33th trial:  20.40449070930481
34th trial:  16.59141206741333
35th trial:  19.562929153442383
36th trial:  18.068703174591064
37th trial:  18.097384452819824
38th trial:  24.080151081085205
39th trial:  23.674723386764526
40th trial:  24.401820182800293
41th trial:  21.359691

   81005    4.0600675e+08   0.000000e+00   5.658126e+08    330s
   81645    4.0402646e+08   0.000000e+00   3.587433e+08    336s
   81965    4.0311192e+08   0.000000e+00   4.303817e+08    341s
   82445    4.0182617e+08   0.000000e+00   2.212090e+08    346s
   82925    4.0036136e+08   0.000000e+00   9.606809e+08    350s
   83405    3.9928051e+08   0.000000e+00   2.062829e+08    356s
   83885    3.9834239e+08   0.000000e+00   5.289306e+08    360s
   84365    3.9724401e+08   0.000000e+00   6.442519e+08    365s
   84845    3.9609009e+08   0.000000e+00   4.717906e+08    371s
   85385    3.9476588e+08   0.000000e+00   6.567191e+08    376s
   85865    3.9335780e+08   0.000000e+00   3.201363e+08    382s
   86345    3.9254636e+08   0.000000e+00   5.644924e+08    386s
   86825    3.9137213e+08   0.000000e+00   3.604831e+08    391s
   87305    3.9022386e+08   0.000000e+00   1.747080e+09    397s
   87625    3.8934259e+08   0.000000e+00   3.079748e+09    400s
   88265    3.8794701e+08   0.000000e+00

...

   76525    4.2086214e+08   0.000000e+00   2.622867e+08    450s
   76845    4.1975775e+08   0.000000e+00   4.833769e+08    456s
   77165    4.1871766e+08   0.000000e+00   2.020589e+08    462s
   77485    4.1809287e+08   0.000000e+00   7.168203e+08    467s
   77645    4.1767158e+08   0.000000e+00   2.133771e+08    470s
   77965    4.1686967e+08   0.000000e+00   1.699350e+08    476s
   78285    4.1594398e+08   0.000000e+00   4.054342e+08    482s
   78445    4.1478479e+08   0.000000e+00   5.604134e+09    485s
   78765    4.1336149e+08   0.000000e+00   3.026398e+08    491s
   79085    4.1227839e+08   0.000000e+00   2.872813e+08    498s
   79245    4.1186775e+08   0.000000e+00   4.139527e+08    500s
   79565    4.1097217e+08   0.000000e+00   9.477676e+08    506s
   79885    4.0978104e+08   0.000000e+00   7.038448e+08    512s
   80045    4.0933194e+08   0.000000e+00   3.305126e+08    516s
   80365    4.0810435e+08   0.000000e+00   2.261591e+08    523s
   80525    4.0741925e+08   0.000000e+00

  107762    3.4955539e+08   0.000000e+00   3.675445e+08   1132s
  108049    3.4860328e+08   0.000000e+00   1.537111e+08   1140s
  108336    3.4798249e+08   0.000000e+00   1.723722e+09   1146s
  108623    3.4734109e+08   0.000000e+00   2.247563e+08   1154s
  108910    3.4683637e+08   0.000000e+00   3.290070e+08   1162s
  109197    3.4628825e+08   0.000000e+00   1.110281e+08   1169s
  109484    3.4593950e+08   0.000000e+00   1.918763e+08   1177s
  109771    3.4543021e+08   0.000000e+00   1.911474e+08   1185s
  110058    3.4502834e+08   0.000000e+00   4.542049e+08   1192s
  110345    3.4463522e+08   0.000000e+00   9.159666e+08   1201s
  110632    3.4425095e+08   0.000000e+00   1.927145e+08   1208s
  110919    3.4403585e+08   0.000000e+00   3.977172e+07   1214s
  111206    3.4398468e+08   0.000000e+00   3.853838e+07   1221s
  111493    3.4393203e+08   0.000000e+00   9.218696e+07   1228s
  111780    3.4388370e+08   0.000000e+00   4.070634e+07   1235s
  112067    3.4383624e+08   0.000000e+00

9th trial:  3.4435367584228516
10th trial:  3.222317695617676
11th trial:  4.427048921585083
12th trial:  6.258956670761108
13th trial:  3.731964349746704
14th trial:  6.006541728973389
15th trial:  3.516818046569824
16th trial:  13.185816287994385
17th trial:  9.42628264427185
18th trial:  8.021140813827515
19th trial:  15.052753210067749
20th trial:  8.601477861404419
21th trial:  13.876555681228638
22th trial:  11.395318031311035
23th trial:  15.770331621170044
24th trial:  8.163666725158691
25th trial:  11.403935670852661
26th trial:  12.654988527297974
27th trial:  8.759099006652832
28th trial:  7.1163482666015625
29th trial:  14.885841369628906
30th trial:  10.327808141708374
31th trial:  26.067309617996216
32th trial:  17.845293045043945
33th trial:  15.600414037704468
34th trial:  11.68530797958374
35th trial:  18.292904138565063
36th trial:  18.3949875831604
37th trial:  24.469829320907593
38th trial:  19.227622747421265
39th trial:  25.07594609260559
40th trial:  18.629470825

   91879    4.0662240e+08   0.000000e+00   4.462699e+08    326s
   92453    4.0607467e+08   0.000000e+00   1.958227e+07    331s
   93027    4.0569746e+08   0.000000e+00   3.039128e+09    336s
   93554    4.0534229e+08   0.000000e+00   5.761477e+08    341s
   94128    4.0487907e+08   0.000000e+00   2.022860e+07    346s
   94702    4.0453359e+08   0.000000e+00   2.511751e+07    351s
   95276    4.0429617e+08   0.000000e+00   2.539572e+07    356s
   95833    4.0398035e+08   0.000000e+00   2.101888e+07    361s
   96407    4.0362488e+08   0.000000e+00   2.290714e+07    365s
   96981    4.0309073e+08   0.000000e+00   3.390613e+08    370s
   97681    4.0265583e+08   0.000000e+00   2.096826e+09    376s
   98255    4.0239534e+08   0.000000e+00   3.261559e+07    382s
   98829    4.0212084e+08   0.000000e+00   3.291066e+07    386s
   99403    4.0180478e+08   0.000000e+00   1.621085e+07    391s
   99977    4.0135480e+08   0.000000e+00   3.583506e+07    396s
  100551    4.0122294e+08   0.000000e+00

7th trial:  3.1702702045440674
8th trial:  2.4017951488494873
9th trial:  2.6645545959472656
10th trial:  2.941316843032837
11th trial:  2.5213217735290527
12th trial:  2.756410837173462
13th trial:  4.563184022903442
14th trial:  2.505640745162964
15th trial:  2.5839359760284424
16th trial:  5.5501604080200195
17th trial:  5.552920818328857
18th trial:  7.886004686355591
19th trial:  5.257276773452759
20th trial:  5.923568964004517
21th trial:  7.445260047912598
22th trial:  6.184189796447754
23th trial:  6.419633626937866
24th trial:  7.9883036613464355
25th trial:  7.208146572113037
26th trial:  6.786082029342651
27th trial:  6.332749128341675
28th trial:  6.060177326202393
29th trial:  9.02234697341919
30th trial:  5.371989011764526
31th trial:  11.344043493270874
32th trial:  11.504116296768188
33th trial:  9.504116535186768
34th trial:  8.74759840965271
35th trial:  11.196488618850708
36th trial:  12.175169229507446
37th trial:  14.25874638557434
38th trial:  13.1252920627594
39t